In [1]:
#imports
import string
import pandas as pd
import pickle as pkl
from collections import Counter
from matplotlib import pyplot as plt

import numpy as np
from random import sample

from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

import sys
sys.path.append('../../src')
import utils

In [2]:
titles = pd.read_pickle('../../data/embeddings/openai-title')
sessions = pd.read_pickle('../../data/datasets/raw-video-level-watches')
videos = pd.read_pickle('../../data/videos_raw_metadata')

In [3]:
upnexts = []
for session in sessions:
    upnext = session['upnext']
    if len(upnext) > 15:
        upnext = upnext[:15]

    if len(upnext) < 15:
        upnext = upnext + sample(list(videos.keys()), 15 - len(upnext))

    # if any of the upnexts are not in the title embeddings, skip
    if not all([u in videos for u in upnext]):
        continue

    upnext = np.array([titles[videos[v]['snippet']['title']] for v in upnext]).flatten()
    upnexts.append(upnext)

In [4]:
upnexts = np.array(upnexts)
upnexts.shape

(6050, 23040)

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(upnexts, upnexts, test_size=0.2)

In [8]:
class Autoencoder(Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
          layers.Dense(23040, activation='relu'),
          layers.Dense(1536, activation='relu'),
          layers.Dense(768, activation='relu'),
          layers.Dense(384, activation='relu'),
          layers.Dense(latent_dim, activation='tanh')
        ])
        
        self.decoder = tf.keras.Sequential([
          layers.Dense(384, activation='relu'),
          layers.Dense(768, activation='relu'),
          layers.Dense(1536, activation='relu'),
          layers.Dense(23040, activation='tanh')
          # layers.Dense(1459, activation='relu'),
          # layers.Dense(14592, activation='tanh')
          # layers.Reshape((-1,768))
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder(128)

autoencoder.compile(optimizer='adam', loss='cosine_similarity', metrics=['accuracy', 'cosine_similarity'])

In [ ]:
autoencoder.fit(X_train, X_train, epochs=5, shuffle=True, batch_size=64, validation_data=(X_test, X_test), verbose=True)

In [35]:
autoencoder.save_weights('../../data/models/autoencoder-upnexts')

In [9]:
autoencoder.load_weights('../../data/models/autoencoder-upnexts')
autoencoder.evaluate(X_test, X_test)

38/38 [==============================] - 20s 447ms/step - loss: -0.8480 - accuracy: 0.0678 - cosine_similarity: 0.8480


[-0.847966194152832, 0.06776859611272812, 0.847966194152832]

In [10]:
sessions = pd.read_pickle('../../data/datasets/raw-video-level-watches')
emb_sessions = []

for session in tqdm(sessions):
    upnext = session['upnext']
    if len(upnext) > 15:
        upnext = upnext[:15]

    if len(upnext) < 15:
        upnext = upnext + sample(list(videos.keys()), 15 - len(upnext))

    # if any of the upnexts are not in the title embeddings, skip
    if not all([u in videos for u in upnext]):
        continue

    upnext = [np.array([titles[videos[v]['snippet']['title']] for v in upnext]).flatten()]
    
    emb_session = dict()
    emb_session['playing'] = session['playing']
    emb_session['upnext'] = session['upnext']
    emb_session['selected'] = session['selected']

    encoded_embs = autoencoder.encoder(upnext[0].reshape(1,-1))
    emb_session['upnext_embs'] = encoded_embs

    emb_sessions.append(emb_session)

100%|██████████| 8563/8563 [17:41<00:00,  8.06it/s]


In [11]:
pkl.dump(emb_sessions, open('../../data/datasets/video-level-watches+autoencoded-upnext', 'wb'))